In [10]:
from tensorflow import keras
import pandas as pd
import numpy as np

In [4]:
model = keras.models.load_model('models/5_accents_international_8k.model')


2023-04-02 12:53:51.974162: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-02 12:53:51.974303: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-02 12:53:51.974347: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-04-02 12:53:51.974388: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-04-02 12:53:51.974456: W tensorf

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4100      
                                                                 
 activation (Activation)     (None, 100)               0         
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               20200     
                                                                 
 activation_1 (Activation)   (None, 200)               0         
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               2

In [6]:
data = pd.read_json('../5_accent_features_8k.json')

In [8]:
data = data.drop(['rms', 'zrc', 'sb', 'sc'], axis=1)
data

lable                                               mfcc
0        american  [[-132.202331543, -106.4561920166, -144.163803...
1        american  [[-120.1662216187, -64.2591247559, -84.6273498...
2        american  [[-204.3710479736, -111.2505950928, -130.19665...
3          indian  [[-204.9865570068, -181.3363800049, -285.47653...
4        american  [[-238.695098877, -131.825668335, -140.2941741...
...           ...                                                ...
11125      indian  [[-392.308380127, -253.5061950684, -204.861923...
11126     british  [[-296.3085021973, -209.2213897705, -214.50019...
11127  australian  [[-158.1745147705, -105.2735824585, -136.48347...
11128    american  [[-106.186164856, -56.3407363892, -119.2368698...
11129      indian  [[-472.9836425781, -234.4741210938, -170.09353...

[11130 rows x 2 columns]

In [11]:
lst = [np.array(l) for l in data.mfcc]
new_lst = []
for l in lst:
    l = np.array([np.mean(values) for values in l])
    new_lst.append(l)
new_lst = np.array(new_lst)
new_lst.shape

(11130, 40)

In [12]:
X=new_lst
y=np.array(data['lable'].tolist())

In [13]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [22]:
test_accuracy=model.evaluate(X, y, verbose=1)
print(test_accuracy[1])

348/348 [==============================] - 0s 1ms/step - loss: 0.0089 - accuracy: 0.9978
0.9978436827659607


In [17]:
help(model.evaluate)

Help on method evaluate in module keras.engine.training:

evaluate(x=None, y=None, batch_size=None, verbose='auto', sample_weight=None, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, return_dict=False, **kwargs) method of keras.engine.sequential.Sequential instance
    Returns the loss value & metrics values for the model in test mode.
    
    Computation is done in batches (see the `batch_size` arg.)
    
    Args:
        x: Input data. It could be:
          - A Numpy array (or array-like), or a list of arrays
            (in case the model has multiple inputs).
          - A TensorFlow tensor, or a list of tensors
            (in case the model has multiple inputs).
          - A dict mapping input names to the corresponding array/tensors,
            if the model has named inputs.
          - A `tf.data` dataset. Should return a tuple
            of either `(inputs, targets)` or
            `(inputs, targets, sample_weights)`.
          - A g